# 3. Розв'язування систем нелінійних рівнянь
------------

Розглянемо нелінійну систему рівнянь

$(1)\qquad\qquad\qquad
   \begin{cases}
    f_1(x_1,\ldots,x_n)=0 \\
    \qquad \ldots \\
    f_n(x_1,\ldots,x_n) = 0
   \end{cases} \quad \Leftrightarrow \quad f(x)=0,
$

де $f_1,\ldots,f_n$ --- визначені на деякій множині $D\subset \mathbb{R}^n$ функції, а 

$\qquad\qquad\qquad
f(x):=\begin{pmatrix}  f_1(x_1,\ldots,x_n) \\
    \ldots \\
    f_n(x_1,\ldots,x_n)
    \end{pmatrix},\ x=\begin{pmatrix}x_1\\ \ldots\\
x_n\end{pmatrix} \in D,\quad \text{ ---  векторна функція}.$

### 3.2.3. Метод Ньютона
--------------
Припускаємо, що $ f_{i}\in C^{1}(D),\, i=\overline{1,n},\, $ і матриця Якобі

$(1)\qquad\qquad\qquad\nabla f(x):=
\begin{pmatrix}
\dfrac{\partial f_{1}(x_1,\ldots,x_n)}{\partial x_{1}} &\ldots &
\dfrac{\partial f_{1}(x_1,\ldots,x_n)}{\partial x_{n}} \\
\vdots &\ddots & \vdots\\
\dfrac{\partial f_{n}(x_1,\ldots,x_n)}{\partial x_{1}} &\ldots &
 \dfrac{\partial f_{n}(x_1,\ldots,x_n)}{\partial x_{n}}
\end{pmatrix}
$

невироджена в кожній точці $x=(x_1,\ldots,x_n)^\top\in D.$

Метод Ньютона визначається так:

* задаємо початкове значення $x^0\in D$ 

* знаходимо послідовні наближення $x^1, x^2, x^3, \ldots$ розв'язку системи рівнянь (1) за формулою

$(2)\qquad\qquad\qquad  x^{k+1}=x^{k}-[\nabla f(x^{k})]^{-1}f(x^{k}), \quad\quad k=0,1,2,....$

Тут $[\nabla f(x^{k})]^{-1}$ -- обернена до матриці Якобі, яку під час реалізації методу будемо знаходити за допомогою бібліотечної функції.


#### Пояснення до використання програмного коду
-----------------
*   Підготувати середовище і потрібні функції : 
    1. виконати комірку для підготовки середовища
    2. виконати комірку, в якій **визначена** функція ``Newton_iteration`` 
     
*   Обчислити чисельний розв'язок конкретної заданої системи
    1. виконати комірку, де **визначені** функції ``f`` і ``inverse_Jacobian_matrix``
    2. задати точність ``eps`` наближеного розв'язку і початкове наближення ``x0``
    3. виконати комірку з **викликом** функції ``Newton_iteration``

#### Програмна реалізація методу
------------

>#### Підготовка середовища

In [23]:
import numpy as np
from scipy import linalg

>#### Допоміжна функція

In [24]:
def norm_3(a):
    """обчислення евклідової норми вектора a"""
    return np.sqrt(np.sum(a**2))

>#### ``Newton_iteration`` -- функція, яка реалізує метод Ньютона 

In [25]:
def Newton_iteration(f,invJ, x0, eps, kmax):
    """ знаходження методом Ньютона наближеного кореня рівняння (1), 
        де f -- непервна функція на відрізку [a, b],  
        f_deriv -- похідна функція на відрізку [a, b]
        x0 -- початкове наближення
        eps -- задана точність
    """   
    x_prev=x0.copy()
    k=1
    
    x_new = x_prev - invJ(x_prev).dot(f(x_prev))
    
    while norm_3(x_new-x_prev) > eps and k<kmax:
        k+=1
        x_prev = x_new
        x_new = x_prev - invJ(x_prev).dot(f(x_prev))       
    return k, x_new

>#### ``f`` і ``inverse_Jacobian_matrix`` -- векторна функція лівої частини рівняння (1) та обернена її матриці Якобі

#### Обчислювальні експерименти
------------

Продемонструємо застосування методу Ньютона до розв'язування нелінійних систем.

**Приклад 1.** (example 7.1) Обчислити методом Ньютона розв'язок системи

$  \left\{
     \begin{array}{ll}
      e^{x_1^2+x_2^2}=1,\\
    e^{x_1^2-x_2^2}=1.
     \end{array}
   \right. 
$

Легко бачити, що її розв'язком є $x=(0, 0)^\top$.

Визначимо векторну функцію, яка задає ліву частину системи, і функцію, яка повертає обернену матрицю для якобіана лівої частини: 

In [26]:
def f(x):
    """функції лівої частини системи рівнянь (1)"""
    f0 = np.exp(x[0]**2 + x[1]**2) - 1
    f1 = np.exp(x[0]**2 - x[1]**2) - 1 
    return np.array([f0,f1])

def inverse_Jacobian_matrix(x):
    """обернена матриця якобіана лівої частини рівняння (1)"""
    df00 = np.exp(x[0]**2 + x[1]**2)*2*x[0]
    df01 = np.exp(x[0]**2 + x[1]**2)*2*x[1]
    df10 = np.exp(x[0]**2 - x[1]**2)*2*x[0]
    df11 =-np.exp(x[0]**2 - x[1]**2)*2*x[1]
    invJ = linalg.inv(np.array([[df00, df01],[df10, df11]]))
    return invJ

Тепер знайдемо чисельний розв'язок при різних значеннях параметра $eps$, попередньо задавши обмеження кількості операцій та початкове наближення:

In [27]:
kmax=100
x0=np.array([0.1, 0.1])

In [28]:
eps=0.001
k, xk = Newton_iteration(f, inverse_Jacobian_matrix, x0, eps,kmax)
print(f"Чисельний розв'язок системи рівнянь x={xk} з точністю eps={eps} за k={k} ітерацій")

Чисельний розв'язок системи рівнянь x=[0.00039585 0.00039585] з точністю eps=0.001 за k=8 ітерацій


In [29]:
eps=0.00001
k, xk = Newton_iteration(f, inverse_Jacobian_matrix, x0, eps,kmax)
print(f"Чисельний розв'язок системи рівнянь x={xk} з точністю eps={eps} за k={k} ітерацій")

Чисельний розв'язок системи рівнянь x=[6.18511692e-06 6.18511692e-06] з точністю eps=1e-05 за k=14 ітерацій


In [30]:
eps=0.0000001
k, xk = Newton_iteration(f, inverse_Jacobian_matrix, x0, eps,kmax)
print(f"Чисельний розв'язок системи рівнянь x={xk} з точністю eps={eps} за k={k} ітерацій")

Чисельний розв'язок системи рівнянь x=[4.82473408e-08 4.82472913e-08] з точністю eps=1e-07 за k=21 ітерацій


Як бачимо, чисельний розв'язок заданої системи рівнянь швидко збігається до точного розв'язку. 

Якщо за початкове наближення взяти вектор правої частини, то для досягнення такої ж точності, як в попередньому випадку, треба виконати більше ітерацій:

In [31]:
x0=np.array([1, 1])

In [32]:
eps=0.001
k, xk = Newton_iteration(f, inverse_Jacobian_matrix, x0, eps,kmax)
print(f"Чисельний розв'язок системи рівнянь x={xk} з точністю eps={eps} за k={k} ітерацій")

Чисельний розв'язок системи рівнянь x=[0.00040068 0.00040068] з точністю eps=0.001 за k=13 ітерацій


In [33]:
eps=0.00001
k, xk = Newton_iteration(f, inverse_Jacobian_matrix, x0, eps,kmax)
print(f"Чисельний розв'язок системи рівнянь x={xk} з точністю eps={eps} за k={k} ітерацій")

Чисельний розв'язок системи рівнянь x=[6.26059772e-06 6.26059770e-06] з точністю eps=1e-05 за k=19 ітерацій


Зазначимо, що при грубшому початковому наближенні матимемо збіжний процес, але, що слід було очікувати,кількість ітерацій значно зростає:

In [34]:
kmax=200
x0=np.array([5, 5])

In [35]:
eps=0.001
k, xk = Newton_iteration(f, inverse_Jacobian_matrix, x0, eps,kmax)
print(f"Чисельний розв'язок системи рівнянь x={xk} з точністю eps={eps} за k={k} ітерацій")

Чисельний розв'язок системи рівнянь x=[0.00036503 0.00036503] з точністю eps=0.001 за k=62 ітерацій


In [36]:
eps=0.00001
k, xk = Newton_iteration(f, inverse_Jacobian_matrix, x0, eps,kmax)
print(f"Чисельний розв'язок системи рівнянь x={xk} з точністю eps={eps} за k={k} ітерацій")

Чисельний розв'язок системи рівнянь x=[5.70352056e-06 5.70352046e-06] з точністю eps=1e-05 за k=68 ітерацій


**Приклад 2.** (приклад 3.8) Обчислити методом Ньютона розв'язок системи

$\left\{
   \begin{array}{rcl}
    4x_1-\sin{x_2}+1&=&0,\\ \\
         \cos{x_1}-2x_2+3&=&0\\
   \end{array}
  \right.
$

Як і в попередньому прикладі, визначимо потрібні функції:

In [37]:
def f(x):
    """функції лівої частини системи рівнянь (1)"""
    f0 = 4*x[0] - np.sin(x[1]) + 1
    f1 = np.cos(x[0]) - 2*x[1] + 3 
    return np.array([f0,f1])

def inverse_Jacobian_matrix(x):
    """обернена матриця якобіана лівої частини рівняння (1)"""
    df00 = 4
    df01 = - np.cos(x[1])
    df10 = - np.sin(x[0])
    df11 =-2
    invJ = linalg.inv(np.array([[df00, df01],[df10, df11]]))
    return invJ

За початкове наближення візьмемо той самий вектор, що і при застосуванні методу простої ітерації:

In [38]:
x0=np.array([-0.25 , 1.5])

Тепер знайдемо чисельний розв'язок при різних значеннях параметра $eps$:

In [39]:
eps=0.01
k, xk = Newton_iteration(f, inverse_Jacobian_matrix, x0, eps,kmax)
print(f"Чисельний розв'язок системи рівнянь x={xk} з точністю eps={eps} за k={k} ітерацій")

Чисельний розв'язок системи рівнянь x=[-0.02266228  1.99987161] з точністю eps=0.01 за k=3 ітерацій


In [40]:
eps=0.0000001
k, xk = Newton_iteration(f, inverse_Jacobian_matrix, x0, eps,kmax)
print(f"Чисельний розв'язок системи рівнянь x={xk} з точністю eps={eps} за k={k} ітерацій")

Чисельний розв'язок системи рівнянь x=[-0.02266229  1.99987161] з точністю eps=1e-07 за k=4 ітерацій


In [41]:
eps=0.000000001
k, xk = Newton_iteration(f, inverse_Jacobian_matrix, x0, eps,kmax)
print(f"Чисельний розв'язок системи рівнянь x={xk} з точністю eps={eps} за k={k} ітерацій")

Чисельний розв'язок системи рівнянь x=[-0.02266229  1.99987161] з точністю eps=1e-09 за k=5 ітерацій


Порівнюючи з методом простої ітерації бачимо, що задана точність досягається за меншу кількість ітерацій.

**Приклад 3.** (приклад 3.9м) Обчислити методом Ньютона розв'язок системи

$  \left\{
     \begin{array}{ll}
      x_1^2-2x_2+3=0,\\
    2x_1+3x_2^2-1=0.
     \end{array}
   \right. 
$

Визначимо векторну функцію, яка задає ліву частину системи, і функцію, яка повертає матрицю якобіана лівої частини: 

In [42]:
def f(x):
    """функції лівої частини системи рівнянь (1)"""
    f0 = x[0]**2- 2*x[1]+3
    f1 = 2*x[0] +3*x[1]**2 - 8 
    return np.array([f0,f1])

def inverse_Jacobian_matrix(x):
    """обернена матриця якобіана лівої частини рівняння (1)"""
    df00 = 2*x[0]
    df01 = -2
    df10 = 2
    df11 = 6*x[1]
    invJ = linalg.inv(np.array([[df00, df01],[df10, df11]]))
    return invJ

In [43]:
eps=0.0001
kmax=10000000
x0=np.array([1.01, 1.99])
k, xk = Newton_iteration(f, inverse_Jacobian_matrix, x0, eps,kmax)
print(f"Чисельний розв'язок системи рівнянь x={xk} з точністю eps={eps} за k={k} ітерацій")

Чисельний розв'язок системи рівнянь x=[0.34762568 1.56042181] з точністю eps=0.0001 за k=5 ітерацій


In [44]:
eps=0.000001
kmax=10000000
x0=np.array([1.01, 1.99])
k, xk = Newton_iteration(f, inverse_Jacobian_matrix, x0, eps,kmax)
print(f"Чисельний розв'язок системи рівнянь x={xk} з точністю eps={eps} за k={k} ітерацій")

Чисельний розв'язок системи рівнянь x=[0.34762568 1.56042181] з точністю eps=1e-06 за k=5 ітерацій
